<a href="https://colab.research.google.com/github/kanyijohn/Deep-Learning-ML-models/blob/main/LLM_Classification_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Explore using pre-trained language models (like BERT, RoBERTa, or others) to generate embeddings for the prompt and responses with More Complex Model Architecture to Consider more sophisticated neural network architectures designed for sequence comparison or ranking. You could fine-tune these models on the preference prediction task. Systematically tune hyperparameters like embedding dimension, maxlen, number of layers, units in dense layers, learning rate, batch size, and number of epochs.

## Install necessary libraries

### Subtask:
Install the `transformers` library to use pre-trained language models.


**Reasoning**:
Install the transformers library using pip.



In [1]:
!pip install transformers

## Data loading

### Subtask:
Load pre-trained model and tokenizer


**Reasoning**:
Import necessary classes and load the pre-trained tokenizer and model.



In [2]:
!pip install kaggle

In [3]:
from google.colab import files
import json

# Upload the file again
uploaded = files.upload()

# Verify the contents (make sure username & key are correct)
for filename in uploaded.keys():
    print(f"Uploaded: {filename}")
    print(uploaded[filename].decode('utf-8'))  # Check the key is correct

Saving kaggle.json to kaggle.json
Uploaded: kaggle.json
{"username":"johnsonkanyi","key":"f8e3cd38e45c532ab64524da20ece09e"}


In [4]:
!mkdir -p ~/.kaggle  # -p prevents error if dir exists
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Restrict permissions

In [5]:
!kaggle competitions list  # Should list competitions (no 401 error)

ref                                                                              deadline             category                reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/arc-prize-2025                               2025-11-03 23:59:00  Featured         1,000,000 Usd        657           False  
https://www.kaggle.com/competitions/google-gemma-3n-hackathon                    2025-08-06 23:59:00  Featured           150,000 Usd          0           False  
https://www.kaggle.com/competitions/make-data-count-finding-data-references      2025-09-09 23:59:00  Research           100,000 Usd        609           False  
https://www.kaggle.com/competitions/map-charting-student-math-misunderstandings  2025-10-15 23:59:00  Featured            55,000 Usd        244           False  
https://www.kaggle.com/compe

In [6]:
!kaggle competitions download -c llm-classification-finetuning

llm-classification-finetuning.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip llm-classification-finetuning.zip

Archive:  llm-classification-finetuning.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [8]:
# @title Step 1: Setup and Data Loading

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, GlobalAveragePooling1D, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re # For text cleaning
from nltk.corpus import stopwords
import nltk
import string

# Download NLTK stopwords (if not already downloaded)
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

# Load the datasets
try:
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    sample_submission_df = pd.read_csv('sample_submission.csv')
    print("Data loaded successfully!")
except FileNotFoundError:
    print("Ensure train.csv, test.csv, and sample_submission.csv are in your Colab environment.")
    # You might need to upload the files or connect to Google Drive
    # from google.colab import files
    # uploaded = files.upload()
    # For Kaggle competitions, the data is usually available in the input directory
    # train_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
    # test_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')
    # sample_submission_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/sample_submission.csv')

print("\nTrain data shape:", train_df.shape)
print("Test data shape:", test_df.shape)
print("\nTrain data head:")
print(train_df.head())
print("\nTest data head:")
print(test_df.head())
print("\nSample submission head:")
print(sample_submission_df.head())

Data loaded successfully!

Train data shape: (57477, 9)
Test data shape: (3, 4)

Train data head:
       id             model_a              model_b  \
0   30192  gpt-4-1106-preview           gpt-4-0613   
1   53567           koala-13b           gpt-4-0613   
2   65089  gpt-3.5-turbo-0613       mistral-medium   
3   96401    llama-2-13b-chat  mistral-7b-instruct   
4  198779           koala-13b   gpt-3.5-turbo-0314   

                                              prompt  \
0  ["Is it morally right to try to have a certain...   
1  ["What is the difference between marriage lice...   
2  ["explain function calling. how would you call...   
3  ["How can I create a test set for a very rare ...   
4  ["What is the best way to travel from Tel-Aviv...   

                                          response_a  \
0  ["The question of whether it is morally right ...   
1  ["A marriage license is a legal document that ...   
2  ["Function calling is the process of invoking ...   
3  ["Creating a 

In [9]:
# @title Step 2: Data Preprocessing and Feature Engineering

# Text Cleaning Function
def clean_text(text):
    text = text.lower() # Convert to lowercase
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text) # Remove punctuation
    # Optional: Remove stopwords
    # stop_words = set(stopwords.words('english'))
    # text = ' '.join([word for word in text.split() if word not in stop_words])
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
    return text

# Apply cleaning to prompt and responses
train_df['prompt_cleaned'] = train_df['prompt'].apply(clean_text)
train_df['response_a_cleaned'] = train_df['response_a'].apply(clean_text)
train_df['response_b_cleaned'] = train_df['response_b'].apply(clean_text)

test_df['prompt_cleaned'] = test_df['prompt'].apply(clean_text)
test_df['response_a_cleaned'] = test_df['response_a'].apply(clean_text)
test_df['response_b_cleaned'] = test_df['response_b'].apply(clean_text)

print("\nText cleaning applied.")
print("\nTrain data with cleaned text:")
print(train_df[['prompt_cleaned', 'response_a_cleaned', 'response_b_cleaned']].head())

# Tokenization and Padding
max_words = 20000 # Maximum number of words to keep based on word frequency
maxlen = 256 # Maximum length of sequences

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")

# Fit tokenizer on combined text from train and test for better vocabulary coverage
all_text = pd.concat([
    train_df['prompt_cleaned'], train_df['response_a_cleaned'], train_df['response_b_cleaned'],
    test_df['prompt_cleaned'], test_df['response_a_cleaned'], test_df['response_b_cleaned']
])
tokenizer.fit_on_texts(all_text)

# Convert text to sequences
train_sequences_prompt = tokenizer.texts_to_sequences(train_df['prompt_cleaned'])
train_sequences_response_a = tokenizer.texts_to_sequences(train_df['response_a_cleaned'])
train_sequences_response_b = tokenizer.texts_to_sequences(train_df['response_b_cleaned'])

test_sequences_prompt = tokenizer.texts_to_sequences(test_df['prompt_cleaned'])
test_sequences_response_a = tokenizer.texts_to_sequences(test_df['response_a_cleaned'])
test_sequences_response_b = tokenizer.texts_to_sequences(test_df['response_b_cleaned'])

# Pad sequences
train_padded_prompt = pad_sequences(train_sequences_prompt, maxlen=maxlen, padding='post', truncating='post')
train_padded_response_a = pad_sequences(train_sequences_response_a, maxlen=maxlen, padding='post', truncating='post')
train_padded_response_b = pad_sequences(train_sequences_response_b, maxlen=maxlen, padding='post', truncating='post')

test_padded_prompt = pad_sequences(test_sequences_prompt, maxlen=maxlen, padding='post', truncating='post')
test_padded_response_a = pad_sequences(test_sequences_response_a, maxlen=maxlen, padding='post', truncating='post')
test_padded_response_b = pad_sequences(test_sequences_response_b, maxlen=maxlen, padding='post', truncating='post')

print("\nText tokenization and padding applied.")
print("\nExample padded sequence (prompt):")
print(train_padded_prompt[0])

# Prepare Target Variable
# We need to represent the winner as a one-hot encoded vector
# [1, 0, 0] for winner_model_a, [0, 1, 0] for winner_model_b, [0, 0, 1] for winner_tie
y_train = train_df[['winner_model_a', 'winner_model_b', 'winner_tie']].values

print("\nTarget variable prepared:")
print(y_train[:5])


Text cleaning applied.

Train data with cleaned text:
                                      prompt_cleaned  \
0  is it morally right to try to have a certain p...   
1  what is the difference between marriage licens...   
2  explain function calling how would you call a ...   
3  how can i create a test set for a very rare ca...   
4  what is the best way to travel from telaviv to...   

                                  response_a_cleaned  \
0  the question of whether it is morally right to...   
1  a marriage license is a legal document that al...   
2  function calling is the process of invoking or...   
3  creating a test set for a very rare category c...   
4  the best way to travel from tel aviv to jerusa...   

                                  response_b_cleaned  
0  as an ai i dont have personal beliefs or opini...  
1  a marriage license and a marriage certificate ...  
2  function calling is the process of invoking a ...  
3  when building a classifier for a very rare cat..

In [10]:
from transformers import BertModel, BertTokenizer

# Load a pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')

print("Pre-trained BERT tokenizer and model loaded successfully.")

Pre-trained BERT tokenizer and model loaded successfully.


## Generate embeddings

### Subtask:
Use the pre-trained model to generate embeddings for the prompt and responses.


**Reasoning**:
Define a function to generate BERT embeddings for text inputs using the loaded tokenizer and model, then apply it to the relevant columns in the training and test dataframes.



In [12]:
import torch
import numpy as np
from tqdm.auto import tqdm

def generate_bert_embeddings(texts, tokenizer, model, max_len=512, batch_size=32):
    """Optimized BERT embedding generation with progress tracking"""
    # Set up device and model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()

    embeddings = []

    # Process in batches with progress bar
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
        batch_texts = texts[i:i + batch_size]

        # Tokenize with automatic padding/truncation
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_len,
            return_tensors="pt"
        ).to(device)

        # Get embeddings - using no_grad for memory efficiency
        with torch.no_grad():
            outputs = model(**inputs)
            # Using mean pooling instead of just CLS token
            last_hidden = outputs.last_hidden_state
            batch_embeddings = torch.mean(last_hidden, dim=1)
            embeddings.append(batch_embeddings.cpu().numpy())

    return np.concatenate(embeddings)

# Generate embeddings with progress tracking
print("Generating embeddings (this may take a few minutes)...")

# Training data embeddings
train_prompt_embeddings = generate_bert_embeddings(
    train_df['prompt_cleaned'].tolist(),
    tokenizer,
    model,
    max_len=256,  # Reduced from 512 for efficiency
    batch_size=64  # Increased batch size
)

train_response_a_embeddings = generate_bert_embeddings(
    train_df['response_a_cleaned'].tolist(),
    tokenizer,
    model,
    max_len=256,
    batch_size=64
)

train_response_b_embeddings = generate_bert_embeddings(
    train_df['response_b_cleaned'].tolist(),
    tokenizer,
    model,
    max_len=256,
    batch_size=64
)

# Test data embeddings
test_prompt_embeddings = generate_bert_embeddings(
    test_df['prompt_cleaned'].tolist(),
    tokenizer,
    model,
    max_len=256,
    batch_size=64
)

test_response_a_embeddings = generate_bert_embeddings(
    test_df['response_a_cleaned'].tolist(),
    tokenizer,
    model,
    max_len=256,
    batch_size=64
)

test_response_b_embeddings = generate_bert_embeddings(
    test_df['response_b_cleaned'].tolist(),
    tokenizer,
    model,
    max_len=256,
    batch_size=64
)

print("\nEmbedding generation complete!")
print(f"Training prompt embeddings shape: {train_prompt_embeddings.shape}")
print(f"Test response embeddings shape: {test_response_a_embeddings.shape}")

Generating embeddings (this may take a few minutes)...


Generating embeddings:   0%|          | 0/899 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/899 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/899 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]


Embedding generation complete!
Training prompt embeddings shape: (57477, 768)
Test response embeddings shape: (3, 768)


**Reasoning**:
The previous command failed because the dataframes `train_df` and `test_df` were not defined in the current session. Need to reload the dataframes and then regenerate the BERT embeddings.



In [15]:
import pandas as pd
import numpy as np
import torch
from transformers import BertModel, BertTokenizer
from tqdm.auto import tqdm
import re
import string

# Text Cleaning Function (optimized)
def clean_text(text):
    text = str(text).lower()
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

# Load data with checks
try:
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    print(f"Data loaded. Train samples: {len(train_df)}, Test samples: {len(test_df)}")

    # Clean text in batches to reduce memory usage
    for col in ['prompt', 'response_a', 'response_b']:
        train_df[f'{col}_cleaned'] = train_df[col].apply(clean_text)
        test_df[f'{col}_cleaned'] = test_df[col].apply(clean_text)

except FileNotFoundError as e:
    print(f"Error loading files: {e}")

# Initialize BERT with memory optimization
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load model with gradient checkpointing to save memory
model = BertModel.from_pretrained(
    'bert-base-uncased',
    torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32
).to(device)
model.eval()

# Memory-optimized embedding generation
def generate_bert_embeddings(texts, tokenizer, model, max_len=128, batch_size=16):
    """Generates embeddings with memory-efficient batch processing"""
    embeddings = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
        batch = texts[i:i+batch_size]

        inputs = tokenizer(
            batch,
            padding='max_length',
            truncation=True,
            max_length=max_len,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            # Use mean pooling for better results than just CLS
            batch_embeds = outputs.last_hidden_state.mean(dim=1)
            embeddings.append(batch_embeds.cpu().numpy())

        # Clear memory explicitly
        del inputs, outputs, batch_embeds
        torch.cuda.empty_cache()

    return np.concatenate(embeddings)

# Generate embeddings with progress tracking
print("\nGenerating embeddings with memory optimization...")

# Adjust these parameters based on your GPU memory
MAX_LEN = 128  # Reduced from 512
BATCH_SIZE = 8  # Start small, increase if memory allows

try:
    # Process training data
    train_prompt_embeddings = generate_bert_embeddings(
        train_df['prompt_cleaned'].tolist(),
        tokenizer,
        model,
        max_len=MAX_LEN,
        batch_size=BATCH_SIZE
    )

    train_response_a_embeddings = generate_bert_embeddings(
        train_df['response_a_cleaned'].tolist(),
        tokenizer,
        model,
        max_len=MAX_LEN,
        batch_size=BATCH_SIZE
    )

    train_response_b_embeddings = generate_bert_embeddings(
        train_df['response_b_cleaned'].tolist(),
        tokenizer,
        model,
        max_len=MAX_LEN,
        batch_size=BATCH_SIZE
    )

    # Process test data
    test_prompt_embeddings = generate_bert_embeddings(
        test_df['prompt_cleaned'].tolist(),
        tokenizer,
        model,
        max_len=MAX_LEN,
        batch_size=BATCH_SIZE
    )

    test_response_a_embeddings = generate_bert_embeddings(
        test_df['response_a_cleaned'].tolist(),
        tokenizer,
        model,
        max_len=MAX_LEN,
        batch_size=BATCH_SIZE
    )

    test_response_b_embeddings = generate_bert_embeddings(
        test_df['response_b_cleaned'].tolist(),
        tokenizer,
        model,
        max_len=MAX_LEN,
        batch_size=BATCH_SIZE
    )

    print("\nEmbedding generation successful!")
    print(f"Train prompt embeddings shape: {train_prompt_embeddings.shape}")
    print(f"Test response embeddings shape: {test_response_a_embeddings.shape}")

except RuntimeError as e:
    print(f"\nMemory error: {e}")
    print("Try reducing MAX_LEN or BATCH_SIZE further")

Data loaded. Train samples: 57477, Test samples: 3

Generating embeddings with memory optimization...


Generating embeddings:   0%|          | 0/7185 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7185 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/7185 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]


Embedding generation successful!
Train prompt embeddings shape: (57477, 768)
Test response embeddings shape: (3, 768)


## Define a more complex model architecture

### Subtask:
Design a neural network architecture that can effectively utilize the generated embeddings for the preference prediction task.

**Reasoning**:
We will define a model architecture that takes the embeddings of the prompt and both responses as inputs. We will then concatenate these embeddings and pass them through dense layers with dropout for regularization. The output layer will have 3 units with a softmax activation function to predict the probability of each outcome (model A wins, model B wins, tie).

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate

# Define the input shapes for the embeddings
embedding_dim = train_prompt_embeddings.shape[1] # Assuming all embeddings have the same dimension

prompt_input = Input(shape=(embedding_dim,), name='prompt_input')
response_a_input = Input(shape=(embedding_dim,), name='response_a_input')
response_b_input = Input(shape=(embedding_dim,), name='response_b_input')

# Concatenate the embeddings
concatenated_embeddings = concatenate([prompt_input, response_a_input, response_b_input])

# Add dense layers
x = Dense(256, activation='relu')(concatenated_embeddings)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer with 3 units for the three classes (model_a_wins, model_b_wins, tie)
output_layer = Dense(3, activation='softmax', name='output_layer')(x)

# Create the model
model = Model(inputs=[prompt_input, response_a_input, response_b_input], outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ prompt_input        │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ response_a_input    │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ response_b_input    │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 2304)      │          0 │ prompt_input[0][… │
│ (Concatenate)       │                   │            │ response_a_input… │
│                     │                   │            │ response_b_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    590,080 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 3)         │        387 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 623,363 (2.38 MB)

 Trainable params: 623,363 (2.38 MB)

 Non-trainable params: 0 (0.00 B)

## Train the model

### Subtask:
Train the new model using the generated embeddings and the target variable.

**Reasoning**:
Split the training data into training and validation sets to monitor performance during training and prevent overfitting. Train the model using the padded sequences and the one-hot encoded target variable.

In [22]:
from sklearn.model_selection import train_test_split
import numpy as np

# First, stack the embeddings properly
# Each of these should have shape (n_samples, embedding_dim)
assert len(train_prompt_embeddings) == len(train_response_a_embeddings) == len(train_response_b_embeddings) == len(y_train)

# Create proper input format for your model
X_prompt_train, X_prompt_val = train_test_split(
    train_prompt_embeddings,
    test_size=0.1,
    random_state=42
)

X_resp_a_train, X_resp_a_val = train_test_split(
    train_response_a_embeddings,
    test_size=0.1,
    random_state=42
)

X_resp_b_train, X_resp_b_val = train_test_split(
    train_response_b_embeddings,
    test_size=0.1,
    random_state=42
)

y_train_split, y_val_split = train_test_split(
    y_train,
    test_size=0.1,
    random_state=42
)

# Now create the proper input dictionaries
X_train_split = {
    'prompt_input': X_prompt_train,
    'response_a_input': X_resp_a_train,
    'response_b_input': X_resp_b_train
}

X_val_split = {
    'prompt_input': X_prompt_val,
    'response_a_input': X_resp_a_val,
    'response_b_input': X_resp_b_val
}

# Verify shapes
print("Training shapes:")
print(f"Prompt: {X_train_split['prompt_input'].shape}")
print(f"Response A: {X_train_split['response_a_input'].shape}")
print(f"Response B: {X_train_split['response_b_input'].shape}")
print(f"Target: {y_train_split.shape}")

print("\nValidation shapes:")
print(f"Prompt: {X_val_split['prompt_input'].shape}")
print(f"Response A: {X_val_split['response_a_input'].shape}")
print(f"Response B: {X_val_split['response_b_input'].shape}")
print(f"Target: {y_val_split.shape}")

# Now train the model
history = model.fit(
    X_train_split,
    y_train_split,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_split, y_val_split),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3),
        tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
    ]
)

Training shapes:
Prompt: (51729, 768)
Response A: (51729, 768)
Response B: (51729, 768)
Target: (51729, 3)

Validation shapes:
Prompt: (5748, 768)
Response A: (5748, 768)
Response B: (5748, 768)
Target: (5748, 3)
Epoch 1/10
1617/1617 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3664 - loss: 1.1148

1617/1617 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.3664 - loss: 1.1148 - val_accuracy: 0.3869 - val_loss: 1.0785
Epoch 2/10
1601/1617 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3822 - loss: 1.0834

1617/1617 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3822 - loss: 1.0834 - val_accuracy: 0.4334 - val_loss: 1.0608
Epoch 3/10
1617/1617 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.3988 - loss: 1.0736 - val_accuracy: 0.4309 - val_loss: 1.0640
Epoch 4/10
1610/1617 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4099 - loss: 1.0697

1617/1617 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.4099 - loss: 1.0697 - val_accuracy: 0.4405 - val_loss: 1.0581
Epoch 5/10
1602/1617 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4183 - loss: 1.0655

1617/1617 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4183 - loss: 1.0655 - val_accuracy: 0.4292 - val_loss: 1.0577
Epoch 6/10
1611/1617 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4204 - loss: 1.0631

1617/1617 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4204 - loss: 1.0631 - val_accuracy: 0.4375 - val_loss: 1.0537
Epoch 7/10
1617/1617 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.4246 - loss: 1.0560 - val_accuracy: 0.4328 - val_loss: 1.0567
Epoch 8/10
1617/1617 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.4238 - loss: 1.0580 - val_accuracy: 0.4403 - val_loss: 1.0559
Epoch 9/10
1617/1617 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4271 - loss: 1.0544 - val_accuracy: 0.4396 - val_loss: 1.0560


## Evaluate the model

### Subtask:
Evaluate the trained model on the validation set using the log loss metric.

**Reasoning**:
Evaluate the trained model on the validation set using the log loss metric, as this is often the evaluation metric used in similar competitions.

In [23]:
from sklearn.metrics import log_loss

# Predict probabilities on the validation set
y_pred_val = model.predict(X_val_split)

# Calculate log loss
logloss_val = log_loss(y_val_split, y_pred_val)

print(f"Validation Log Loss: {logloss_val}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Validation Log Loss: 1.0560339828939243


## Hyperparameter Tuning

### Subtask:
Systematically tune hyperparameters of the model and training process to improve performance.

**Reasoning**:
Hyperparameter tuning is a crucial step to optimize model performance. We will use KerasTuner to perform a random search over a defined search space for hyperparameters such as the number of dense layers, units in dense layers, dropout rate, learning rate, and batch size.

In [25]:
# First install required packages
!pip install keras-tuner -q
!pip install tensorflow -q

import keras_tuner as kt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

def build_model(hp):
    embedding_dim = train_prompt_embeddings.shape[1]

    # Input layers
    prompt_input = Input(shape=(embedding_dim,), name='prompt_input')
    response_a_input = Input(shape=(embedding_dim,), name='response_a_input')
    response_b_input = Input(shape=(embedding_dim,), name='response_b_input')

    # Concatenate embeddings
    merged = concatenate([prompt_input, response_a_input, response_b_input])

    # Tune number of layers and units
    x = merged
    for i in range(hp.Int('num_layers', 1, 3)):
        x = Dense(
            units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=32),
            activation='relu'
        )(x)
        x = Dropout(hp.Float(f'dropout_{i}', 0.0, 0.5, step=0.1))(x)

    # Output layer
    output = Dense(3, activation='softmax')(x)

    # Create model
    model = Model(
        inputs=[prompt_input, response_a_input, response_b_input],
        outputs=output
    )

    # Tune learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Prepare data for tuning (using dictionary format)
X_train = {
    'prompt_input': train_prompt_embeddings,
    'response_a_input': train_response_a_embeddings,
    'response_b_input': train_response_b_embeddings
}
y_train = train_df[['winner_model_a', 'winner_model_b', 'winner_tie']].values

# Create train/validation split
indices = np.arange(len(train_prompt_embeddings))
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

X_train_tune = {
    'prompt_input': train_prompt_embeddings[train_idx],
    'response_a_input': train_response_a_embeddings[train_idx],
    'response_b_input': train_response_b_embeddings[train_idx]
}

X_val_tune = {
    'prompt_input': train_prompt_embeddings[val_idx],
    'response_a_input': train_response_a_embeddings[val_idx],
    'response_b_input': train_response_b_embeddings[val_idx]
}

y_train_tune = y_train[train_idx]
y_val_tune = y_train[val_idx]

# Initialize tuner
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_tuning_dir',
    project_name='llm_preference_tuning'
)

# Start the search
tuner.search(
    X_train_tune,
    y_train_tune,
    epochs=10,
    validation_data=(X_val_tune, y_val_tune),
    batch_size=32
)

# Get best model
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters()[0]

print("\nBest Hyperparameters Found:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

# Evaluate best model
val_loss, val_acc = best_model.evaluate(X_val_tune, y_val_tune)
print(f"\nBest Model Validation Accuracy: {val_acc:.4f}")
print(f"Best Model Validation Loss: {val_loss:.4f}")

Trial 10 Complete [00h 01m 11s]
val_accuracy: 0.4507654905319214

Best val_accuracy So Far: 0.45633262395858765
Total elapsed time: 00h 11m 15s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Best Hyperparameters Found:
num_layers: 3
units_0: 480
dropout_0: 0.1
learning_rate: 0.0001
units_1: 288
dropout_1: 0.1
units_2: 480
dropout_2: 0.30000000000000004
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4606 - loss: 1.0364

Best Model Validation Accuracy: 0.4563
Best Model Validation Loss: 1.0393


In [27]:
# Calculate log loss
logloss_val = log_loss(y_val_split, y_pred_val)

print(f"Validation Log Loss: {logloss_val}")

Validation Log Loss: 1.0560339828939243


## Make predictions on test data

### Subtask:
Use the best performing model to make predictions on the test data.

**Reasoning**:
Use the best model found during hyperparameter tuning to predict the probabilities on the test dataset.

In [26]:
# Prepare test data for prediction
X_test = [test_prompt_embeddings, test_response_a_embeddings, test_response_b_embeddings]

# Make predictions on the test data using the best model
test_predictions = best_model.predict(X_test)

print("Test predictions generated.")
print("Shape of test predictions:", test_predictions.shape)
print("Sample test predictions:\n", test_predictions[:5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 665ms/step
Test predictions generated.
Shape of test predictions: (3, 3)
Sample test predictions:
 [[0.27163264 0.2395895  0.48877785]
 [0.22848147 0.39324048 0.37827805]
 [0.31398875 0.39510003 0.29091117]]


## Make predictions on test data

### Subtask:
Use the best performing model to make predictions on the test data.

**Reasoning**:
Use the best model found during hyperparameter tuning to predict the probabilities on the test dataset.

In [28]:
# Prepare test data for prediction
X_test = [test_prompt_embeddings, test_response_a_embeddings, test_response_b_embeddings]

# Make predictions on the test data using the best model
test_predictions = best_model.predict(X_test)

print("Test predictions generated.")
print("Shape of test predictions:", test_predictions.shape)
print("Sample test predictions:\n", test_predictions[:5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Test predictions generated.
Shape of test predictions: (3, 3)
Sample test predictions:
 [[0.27163264 0.2395895  0.48877785]
 [0.22848147 0.39324048 0.37827805]
 [0.31398875 0.39510003 0.29091117]]


## Generate submission file

### Subtask:
Create a submission file in the specified format.

**Reasoning**:
Create a submission file in the format required by the competition, which is a CSV file with the columns `id`, `winner_model_a`, `winner_model_b`, and `winner_tie`.

In [29]:
# Create the submission DataFrame
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'winner_model_a': test_predictions[:, 0],
    'winner_model_b': test_predictions[:, 1],
    'winner_tie': test_predictions[:, 2]
})

# Save the submission file
submission_df.to_csv('submission.csv', index=False)

print("Submission file created successfully!")
print("Submission file head:")
print(submission_df.head())

Submission file created successfully!
Submission file head:
        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.271633        0.239589    0.488778
1   211333        0.228481        0.393240    0.378278
2  1233961        0.313989        0.395100    0.290911


## Finish task

Summarize the results and present the submission file.

**Reasoning**:
Summarize the model performance based on the validation log loss and confirm the submission file has been generated.

**Results**:
The model training and hyperparameter tuning have been completed.
The validation log loss for the best model found during tuning is: {logloss_best_val:.4f}

A submission file named `submission.csv` has been generated in the current directory. This file contains the predicted probabilities for the test set in the required format.

# Task
Explore advanced text representation using pre-trained language models (like BERT, RoBERTa, or others) to generate embeddings for the prompt and responses. Consider more sophisticated neural network architectures designed for sequence comparison or ranking. Fine-tune these models on the preference prediction task. Systematically tune hyperparameters like embedding dimension, maxlen, number of layers, units in dense layers, learning rate, batch size, and number of epochs. Define a complex model architecture, train the model, evaluate the model using log loss, perform hyperparameter tuning, and make predictions on test data. Fine-tune the notebook to achieve a better log loss close to 0. Explore all the above steps.

## More extensive hyperparameter tuning

### Subtask:
Increase the number of trials and potentially the search space for the current model architecture using KerasTuner to find better hyperparameters.


**Reasoning**:
Increase the number of trials for the hyperparameter search and potentially expand the search space to find a better model configuration. Then, evaluate the best model found.



In [32]:
# Increase the number of trials and potentially expand the search space
MAX_TRIALS = 2 # Increased from 10

def build_model_improved(hp):
    embedding_dim = train_prompt_embeddings.shape[1]

    # Input layers
    prompt_input = Input(shape=(embedding_dim,), name='prompt_input')
    response_a_input = Input(shape=(embedding_dim,), name='response_a_input')
    response_b_input = Input(shape=(embedding_dim,), name='response_b_input')

    # Concatenate embeddings
    merged = concatenate([prompt_input, response_a_input, response_b_input])

    # Tune number of layers and units
    x = merged
    for i in range(hp.Int('num_layers', 1, 4)): # Increased max layers to 4
        x = Dense(
            units=hp.Int(f'units_{i}', min_value=128, max_value=768, step=64), # Increased unit range
            activation=hp.Choice(f'activation_{i}', values=['relu', 'tanh']) # Added activation tuning
        )(x)
        x = Dropout(hp.Float(f'dropout_{i}', 0.1, 0.6, step=0.1))(x) # Adjusted dropout range

    # Output layer
    output = Dense(3, activation='softmax')(x)

    # Create model
    model = Model(
        inputs=[prompt_input, response_a_input, response_b_input],
        outputs=output
    )

    # Tune learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[5e-3, 1e-3, 5e-4, 1e-4]) # Added more learning rates

    model.compile(
        optimizer=Adam(learning_rate=hp_learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Initialize tuner with increased trials and potentially expanded search space
tuner_improved = kt.RandomSearch(
    hypermodel=build_model_improved,
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    executions_per_trial=1,
    directory='my_tuning_dir_improved',
    project_name='llm_preference_tuning_improved'
)

# Start the search with the updated parameters
tuner_improved.search(
    X_train_tune,
    y_train_tune,
    epochs=15, # Increased epochs
    validation_data=(X_val_tune, y_val_tune),
    batch_size=64 # Increased batch size if memory allows
)

# Get best model and hyperparameters
best_model_improved = tuner_improved.get_best_models(num_models=1)[0]
best_hps_improved = tuner_improved.get_best_hyperparameters()[0]

print("\nBest Hyperparameters Found (Improved Search):")
for param, value in best_hps_improved.values.items():
    print(f"{param}: {value}")

# Evaluate the best model
val_loss_improved, val_acc_improved = best_model_improved.evaluate(X_val_tune, y_val_tune)
print(f"\nBest Model (Improved Search) Validation Accuracy: {val_acc_improved:.4f}")
print(f"Best Model (Improved Search) Validation Loss: {val_loss_improved:.4f}")

# Calculate and print Log Loss for the best model
y_pred_val_improved = best_model_improved.predict(X_val_tune)
logloss_val_improved = log_loss(y_val_tune, y_pred_val_improved)
print(f"Best Model (Improved Search) Validation Log Loss: {logloss_val_improved:.4f}")

Reloading Tuner from my_tuning_dir_improved/llm_preference_tuning_improved/tuner0.json


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Best Hyperparameters Found (Improved Search):
num_layers: 3
units_0: 192
activation_0: relu
dropout_0: 0.5
learning_rate: 0.0001
units_1: 128
activation_1: relu
dropout_1: 0.1
units_2: 128
activation_2: relu
dropout_2: 0.1
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4628 - loss: 1.0339

Best Model (Improved Search) Validation Accuracy: 0.4561
Best Model (Improved Search) Validation Loss: 1.0381
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Best Model (Improved Search) Validation Log Loss: 1.0381


## More extensive hyperparameter tuning

### Subtask:
Increase the number of trials and potentially the search space for the current model architecture using KerasTuner to find better hyperparameters.

**Reasoning**:
Increase the number of trials for the hyperparameter search and potentially expand the search space to find a better model configuration. Then, evaluate the best model found.

## Experiment with Different Pre-trained Models

### Subtask:
Load and generate embeddings using a different pre-trained language model (e.g., RoBERTa).

**Reasoning**:
Load the RoBERTa tokenizer and model from the `transformers` library. Define a function to generate embeddings using this model, similar to the BERT embedding function, and apply it to the cleaned text data.

In [ ]:
from transformers import RobertaModel, RobertaTokenizer
import torch
from tqdm.auto import tqdm
import numpy as np

# Load a pre-trained RoBERTa tokenizer and model
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaModel.from_pretrained('roberta-base')

print("Pre-trained RoBERTa tokenizer and model loaded successfully.")

def generate_roberta_embeddings(texts, tokenizer, model, max_len=128, batch_size=16):
    """Generates RoBERTa embeddings for a list of text inputs in batches."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()  # Set model to evaluation mode
    embeddings = []

    # Process in batches with progress bar
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating RoBERTa embeddings"):
        batch_texts = texts[i:i + batch_size]
        encoded_inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_len,
            return_tensors='pt'  # Return PyTorch tensors
        )

        encoded_inputs = {key: val.to(device) for key, val in encoded_inputs.items()}

        with torch.no_grad():
            outputs = model(**encoded_inputs)

        # Using mean pooling of the last hidden state
        batch_embeddings = torch.mean(outputs.last_hidden_state, dim=1)
        embeddings.append(batch_embeddings.cpu().numpy())

        # Clear memory explicitly
        del encoded_inputs, outputs, batch_embeddings
        torch.cuda.empty_cache()


    return np.concatenate(embeddings)

# Generate embeddings for training data using RoBERTa
print("\nGenerating RoBERTa embeddings for training data...")
train_prompt_embeddings_roberta = generate_roberta_embeddings(
    train_df['prompt_cleaned'].tolist(),
    roberta_tokenizer,
    roberta_model,
    max_len=MAX_LEN, # Using the same max_len as before
    batch_size=BATCH_SIZE # Using the same batch_size as before
)

train_response_a_embeddings_roberta = generate_roberta_embeddings(
    train_df['response_a_cleaned'].tolist(),
    roberta_tokenizer,
    roberta_model,
    max_len=MAX_LEN,
    batch_size=BATCH_SIZE
)

train_response_b_embeddings_roberta = generate_roberta_embeddings(
    train_df['response_b_cleaned'].tolist(),
    roberta_tokenizer,
    roberta_model,
    max_len=MAX_LEN,
    batch_size=BATCH_SIZE
)
print("Training RoBERTa embeddings generated.")

# Generate embeddings for test data using RoBERTa
print("\nGenerating RoBERTa embeddings for test data...")
test_prompt_embeddings_roberta = generate_roberta_embeddings(
    test_df['prompt_cleaned'].tolist(),
    roberta_tokenizer,
    roberta_model,
    max_len=MAX_LEN,
    batch_size=BATCH_SIZE
)

test_response_a_embeddings_roberta = generate_roberta_embeddings(
    test_df['response_a_cleaned'].tolist(),
    roberta_tokenizer,
    roberta_model,
    max_len=MAX_LEN,
    batch_size=BATCH_SIZE
)

test_response_b_embeddings_roberta = generate_roberta_embeddings(
    test_df['response_b_cleaned'].tolist(),
    roberta_tokenizer,
    roberta_model,
    max_len=MAX_LEN,
    batch_size=BATCH_SIZE
)
print("Test RoBERTa embeddings generated.")

print("\nShape of training prompt embeddings (RoBERTa):", train_prompt_embeddings_roberta.shape)
print("Shape of training response_a embeddings (RoBERTa):", train_response_a_embeddings_roberta.shape)
print("Shape of training response_b embeddings (RoBERTa):", train_response_b_embeddings_roberta.shape)
print("\nShape of test prompt embeddings (RoBERTa):", test_prompt_embeddings_roberta.shape)
print("Shape of test response_a embeddings (RoBERTa):", test_response_a_embeddings_roberta.shape)
print("Shape of test response_b embeddings (RoBERTa):", test_response_b_embeddings_roberta.shape)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pre-trained RoBERTa tokenizer and model loaded successfully.

Generating RoBERTa embeddings for training data...


Generating RoBERTa embeddings:   0%|          | 0/7185 [00:00<?, ?it/s]